In [1]:
import pickle
import numpy as np
import netCDF4 as nc

In [2]:
data = np.array(nc.Dataset("file_path to the concatenated nc4file")["precipitation"][:])
#data.shape should be 6940*1440*400

In [ ]:
days_ERE = [[]]*1440
for k in range(1440):
    days_ERE[k] = [None] *400
# at each location, choose 95 percentile of all values that are greater than 1
for i in range(1440):
    for j in range(400):
        data_grid = data[:, i, j]  
        filtered_data_grid = data_grid[data_grid>1]
        if len(filtered_data_grid) > 0:
            percentile_of_grid = np.percentile(filtered_data_grid, 95)
            days = np.where(data_grid >= percentile_of_grid)[0]
            if type(days)==int:
                days_ERE[i][j] = [days]
            else:
                days_ERE[i][j] = days
        else:
            days_ERE[i][j] = 0

In [ ]:
start_all_days_ERE = [[]]*1440
for k in range(1440):
    start_all_days_ERE[k] = [None] *400

#delete consecutive dates
for i in range(1440):
    for j in range(400):
        ERE_local = days_ERE[i][j]
        if type(ERE_local) == int:
            start_all_days_ERE[i][j] = ERE_local
        else:
            start_day_local = []
            prev_date = None
            for day in ERE_local:
                if prev_date is not None and day == prev_date + 1:
                    prev_date = day
                    continue
                start_day_local.append(day)
                prev_date = day
            start_all_days_ERE[i][j] = start_day_local

In [ ]:
with open("ERE_start_all_days", "wb") as fp:   #Pickling
   pickle.dump(start_all_days_ERE, fp) 